# Homework 2


Columns represent (in order):
1) Company where they work
2) Job title
3) Office location
4) Total annual compensation (in $)
5) Base salary (in $)
6) Value of stock grants (in $)
7) Bonus payments (in $)
8) Years of relevant experience (in years)
9) Time with this company (in years)
10) Gender (self-reported)
11) Terminal Degree is Masters (1 = yes)
12) Terminal Degree is Bachelors (1 = yes)
13) Terminal Degree is Doctorate (1 = yes)
14) Terminal Degree is High School (1 = yes)
15) Terminal Degree is some college (1 = yes)
16) Self-identifies as Asian (1 = yes)
17) Self-identifies as White (1 = yes)
18) Self-identifies as Multi-Racial (1 = yes)
19) Self-identifies as Black (1 = yes)
20) Self-identifies as Hispanic (1 = yes)
21) Race as a qualitative variable
22) Education as a qualitative variable
23) Age (in years)
24) Height (in inches)
25) Zodiac sign (Tropical calendar, 1 = Aries, 12 = Pisces, with everything else in between)
26) SAT score
27) GPA
We/you will want to use most of these variables in prediction models.
This data is self-reported, so sometimes it will be missing if the person (for whatever reason) did not provide this information. For instance, the information on education (variables 11-15) is only meaningfully interpretable for any given row, if the corresponding value in variable 22 is not “NA”. NA indicates missing data. The same is true for variables 16 to 21.

## Loading the data:

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("techSalaries2017.csv")

In [2]:
data

,company,title,location,totalyearlycompensation,basesalary,stockgrantvalue,bonus,yearsofexperience,yearsatcompany,gender,...,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education,Age,Height,Zodiac,SAT,GPA
0,Oracle,Product Manager,"Redwood City, CA",127000,107000,20000.0,10000.0,1.5,1.5,NaN,...,0,0,0,NaN,NaN,38,68.590,5,829,2.13
1,eBay,Software Engineer,"San Francisco, CA",100000,0,0.0,0.0,5.0,3.0,NaN,...,0,0,0,NaN,NaN,34,68.059,7,993,2.77
2,Amazon,Product Manager,"Seattle, WA",310000,155000,0.0,0.0,8.0,0.0,NaN,...,0,0,0,NaN,NaN,42,61.343,3,1200,3.26
3,Apple,Software Engineering Manager,"Sunnyvale, CA",372000,157000,180000.0,35000.0,7.0,5.0,NaN,...,0,0,0,NaN,NaN,28,66.495,3,1170,3.07
4,Microsoft,Software Engineer,"Mountain View, CA",157000,0,0.0,0.0,5.0,3.0,NaN,...,0,0,0,NaN,NaN,34,69.015,2,1115,2.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62637,Google,Software Engineer,"Seattle, WA",327000,155000,150000.0,22000.0,10.0,1.0,NaN,...,0,0,0,NaN,NaN,52,64.792,6,1064,2.76
62638,Microsoft,Software Engineer,"Redmond, WA",237000,146900,73200.0,16000.0,2.0,2.0,NaN,...,0,0,0,NaN,NaN,21,69.483,9,1160,3.27
62639,MSFT,Software Engineer,"Seattle, WA",220000,157000,25000.0,20000.0,14.0,12.0,NaN,...,0,0,0,NaN,NaN,43,62.331,12,1014,2.78
62640,Salesforce,Software Engineer,"San Francisco, CA",280000,194688,57000.0,29000.0,8.0,4.0,NaN,...,0,0,0,NaN,NaN,48,70.357,10,1109,3.04


## 1. Using multiple linear regression: What is the best predictor of total annual compensation, how much variance is explained by this predictor vs. the full multiple regression model?


## 2. Using ridge regression to do the same as in 1): How does the model change or improve compared to OLS? What is the optimal lambda?


## 3. Using Lasso regression to do the same as in 1): How does the model change now? How many of the predictor betas are shrunk to exactly 0? What is the optimal lambda now?

## 4. There is controversy as to the existence of a male/female gender pay gap in tech job compensation. Build a logistic regression model (with gender as the outcome variable) to see if there is an appreciable beta associated with total annual compensation with and without controlling for other factors.

## 5. Build a logistic regression model to see if you can predict high and low pay from years of relevant experience, age, height, SAT score and GPA, respectively.


## Extra credit:
## a) Is salary, height or age normally distributed? Does this surprise you? Why or why not?
## b) Tell us something interesting about this dataset that is not already covered by the questions above and that is not obvious.